In [64]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

In [65]:
df=pd.read_excel('Book01.xlsx')
df.head()

,Date,MB033,MB034,MB035,MB036
0,2021-01-01,site off,4,71,40
1,2021-01-02,88,66,52,12
2,2021-01-03,52,40,54,72
3,2021-01-04,49,34,32,28
4,2021-01-05,82,32,85,49


In [66]:
df.tail()

,Date,MB033,MB034,MB035,MB036
495,2022-05-11,70,42,6,2
496,2022-05-12,92,3,99,64
497,2022-05-13,84,82,24,30
498,2022-05-14,98,50,19,60
499,2022-05-15,3,0,66,48


In [67]:
#check for missing values in train data
df.isnull().sum()
 #No missing valuues



Date     0
MB033    0
MB034    0
MB035    0
MB036    0
dtype: int64

Here for better understanding of the data, We can eloborate as month and weekday wise.

In [68]:
df['MB034']

0       4
1      66
2      40
3      34
4      32
       ..
495    42
496     3
497    82
498    50
499     0
Name: MB034, Length: 500, dtype: object

In [69]:
df['MB033'] = df['MB033'].replace(['site off'],0)
df['MB034'] = df['MB034'].replace(['site off'],0)
df['MB035'] = df['MB035'].replace(['site off'],0)
df['MB036'] = df['MB036'].replace(['site off'],0)









In [70]:
arr1 = df['MB034'].values
arr1

array([  4,  66,  40,  34,  32,  77,  15,  36,  64,  92,   5,   6,  71,
        87,   8,  32,  59,  55,  47,  82,  48,  95,  74,  66,  49,  40,
        20,  62,  18,  84,   0,  91,   7,  65,  65,  22,  74,  30,  66,
        45,  41,  20,  27,  19,  73,  32,  90,  48,  17,  59,  30,  28,
        32,  61,  38,  73,  48,  38,   0,  26,   5,  22,  34,  39,  91,
        51,  32,  94,  34,  71,   4,  69,  93,  61,  58,  68,  96,  83,
        92,  43,  31,   1,  59,  90,  46,  99,  31,  53,  25,   0,  65,
        25,   9,  30,  38,   4,  46,  12,  24,  61,  27,  29,  48,  90,
        51,  78,  87,  90,  24,  57,  24,   2,  44,  69,  22,  21,  18,
        44,  62,   0,  69,  59,  86,  59,  25,  68,  18,  68,  76,  14,
        99,  79,  12,  20,  65,  30,  76,  51,  33,  76,  94,  89,  61,
        53,  87,  90,  39,  14,  79,  16,   0,  66,  42,  55,  33,  82,
        57,  40,  55,  99,  25,   6,  98,  41,  93,  34,   6,  84,  34,
        75,  31,  75,  99,  57,  36,  97,  82,  17,  31,  81,   

In [71]:
import numpy as np
def splitSequence(seq, n_steps):
    
    #Declare X and y as empty list
    X = []
    y = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        pass
    #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)
    
    return X,y 
    
    pass

n_steps = 20
X, y = splitSequence(arr1, n_steps = 20)

In [72]:
X.shape,y.shape

((480, 20), (480,))

In [73]:
for i in range(len(X)):
    print(X[i], y[i])

[ 4 66 40 34 32 77 15 36 64 92  5  6 71 87  8 32 59 55 47 82] 48
[66 40 34 32 77 15 36 64 92  5  6 71 87  8 32 59 55 47 82 48] 95
[40 34 32 77 15 36 64 92  5  6 71 87  8 32 59 55 47 82 48 95] 74
[34 32 77 15 36 64 92  5  6 71 87  8 32 59 55 47 82 48 95 74] 66
[32 77 15 36 64 92  5  6 71 87  8 32 59 55 47 82 48 95 74 66] 49
[77 15 36 64 92  5  6 71 87  8 32 59 55 47 82 48 95 74 66 49] 40
[15 36 64 92  5  6 71 87  8 32 59 55 47 82 48 95 74 66 49 40] 20
[36 64 92  5  6 71 87  8 32 59 55 47 82 48 95 74 66 49 40 20] 62
[64 92  5  6 71 87  8 32 59 55 47 82 48 95 74 66 49 40 20 62] 18
[92  5  6 71 87  8 32 59 55 47 82 48 95 74 66 49 40 20 62 18] 84
[ 5  6 71 87  8 32 59 55 47 82 48 95 74 66 49 40 20 62 18 84] 0
[ 6 71 87  8 32 59 55 47 82 48 95 74 66 49 40 20 62 18 84  0] 91
[71 87  8 32 59 55 47 82 48 95 74 66 49 40 20 62 18 84  0 91] 7
[87  8 32 59 55 47 82 48 95 74 66 49 40 20 62 18 84  0 91  7] 65
[ 8 32 59 55 47 82 48 95 74 66 49 40 20 62 18 84  0 91  7 65] 65
[32 59 55 47 82 48 95 74 66

In [74]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X[:2])

[[[ 4]
  [66]
  [40]
  [34]
  [32]
  [77]
  [15]
  [36]
  [64]
  [92]
  [ 5]
  [ 6]
  [71]
  [87]
  [ 8]
  [32]
  [59]
  [55]
  [47]
  [82]]

 [[66]
  [40]
  [34]
  [32]
  [77]
  [15]
  [36]
  [64]
  [92]
  [ 5]
  [ 6]
  [71]
  [87]
  [ 8]
  [32]
  [59]
  [55]
  [47]
  [82]
  [48]]]


In [75]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [76]:
model2 = tf.keras.Sequential()
model2.add(layers.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model2.add(layers.Dense(1))

In [77]:
model2.layers

In [78]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [79]:
model2.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [80]:
model2.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
15/15 [==============================] - 1s 10ms/step - loss: 2439.8884 - accuracy: 0.0167
Epoch 2/200
15/15 [==============================] - 0s 11ms/step - loss: 960.0922 - accuracy: 0.0125
Epoch 3/200
15/15 [==============================] - 0s 11ms/step - loss: 945.9952 - accuracy: 0.0125
Epoch 4/200
15/15 [==============================] - 0s 11ms/step - loss: 960.4709 - accuracy: 0.0125
Epoch 5/200
15/15 [==============================] - 0s 11ms/step - loss: 942.7667 - accuracy: 0.0125
Epoch 6/200
15/15 [==============================] - 0s 11ms/step - loss: 908.6396 - accuracy: 0.0125
Epoch 7/200
15/15 [==============================] - 0s 11ms/step - loss: 930.9518 - accuracy: 0.0125
Epoch 8/200
15/15 [==============================] - 0s 11ms/step - loss: 84171472896.0000 - accuracy: 0.0125
Epoch 9/200
15/15 [==============================] - 0s 11ms/step - loss: 99001016.0000 - accuracy: 0.0188
Epoch 10/200
15/15 [==============================] - 0s 10ms/step -

In [81]:
test_data = arr1[-101:-1]
test_data

array([67, 91, 91, 43, 62, 67, 97, 96, 87, 18, 10, 18, 78, 55, 90, 28, 92,
       67, 16, 14, 97, 51, 97, 55,  0, 60,  9, 58, 81,  4, 16, 57, 30, 91,
        5, 98, 64, 14, 72, 57, 85, 27, 80, 16,  1,  1,  8, 61, 54, 26, 23,
       52, 73, 81, 47,  0, 22, 62,  1, 63, 30, 40, 40, 48, 42, 43, 28, 57,
       53, 22, 93, 17, 55, 20, 63, 91, 63, 28, 49,  2, 94, 65, 84, 63, 42,
        0, 29, 12, 99, 33, 53, 72, 68,  9, 69, 19, 42,  3, 82, 50])

In [82]:
test_data = arr1[-20:]
test_data = test_data.reshape((1, n_steps, n_features))
test_data

array([[[65],
        [84],
        [63],
        [42],
        [ 0],
        [29],
        [12],
        [99],
        [33],
        [53],
        [72],
        [68],
        [ 9],
        [69],
        [19],
        [42],
        [ 3],
        [82],
        [50],
        [ 0]]])

In [83]:
predictNextNumber = model2.predict(test_data, verbose=1)
print(predictNextNumber)

1/1 [==============================] - 0s 183ms/step
[[35.850235]]
